# `scapy`: Sniffing Network Packets

`scapy` is a python package for dissecting and implementing network protocols.

Full documentation can be found at: http://www.secdev.org/projects/scapy/doc/

## 1. Importing `scapy`

The first 2 lines of the next cell suppress warnings from the `scapy` package.

In [1]:
import logging
logging.getLogger("scapy.runtime").setLevel(logging.ERROR)
from scapy.all import *

## 2. Sniffing packets

`scapy` can sniff packets off the wire and process and display them.

The filter is a string in the Berkeley packet filter style: http://biot.com/capstats/bpf.html.

The `nsummary()` method prints a summary of each packet,
along with its index in the collection.

The next cell will sniff packets for 2 seconds and spit out the packet trace.

In [2]:
import signal, time

def time_out(signum, frame):
    raise KeyboardInterrupt

signal.signal(signal.SIGALRM, time_out)
signal.alarm(2)

try:
    pkt_trace = sniff(iface="br0", filter="udp or arp")
except KeyboardInterrupt:
    pass
pkt_trace.nsummary()

0000 Ether / ARP who has 172.19.73.222 says 172.19.74.235 / Padding
0001 Ether / IP / UDP 172.19.73.194:netbios_ns > 172.19.75.255:netbios_ns / NBNSQueryRequest
0002 Ether / IPv6 / UDP fe80::262:ecff:fe8d:e53f:36228 > ff02::c:1900 / Raw
0003 Ether / IP / UDP 172.19.72.146:netbios_ns > 172.19.75.255:netbios_ns / NBNSQueryRequest
0004 Ether / IP / UDP 172.19.73.42:netbios_ns > 172.19.75.255:netbios_ns / NBNSQueryRequest
0005 Ether / IPv6 / UDP fe80::262:ecff:fe8d:e53f:36228 > ff02::c:1900 / Raw
0006 Ether / IP / UDP 172.19.73.42:netbios_ns > 172.19.75.255:netbios_ns / NBNSQueryRequest
0007 Ether / IP / UDP 172.19.72.226:netbios_ns > 172.19.75.255:netbios_ns / NBNSQueryRequest
0008 Ether / IP / UDP 172.19.72.192:54915 > 172.19.75.255:54915 / Raw
0009 Ether / ARP who has 172.19.76.189 says 0.0.0.0 / Padding
0010 Ether / IP / UDP 172.19.73.240:netbios_ns > 172.19.75.255:netbios_ns / NBNSQueryRequest
0011 Ether / ARP who has 172.19.3.254 says 172.19.2.68 / Padding
0012 Ether / IP / UDP 172.1

If you execute the previous code on a busy network, you'll probably see lots of packets which are not MQTTSN packets.
The following code separates the MQTTSN packets from the other 'uninteresting' packets.
In this code x[MQTTSN] returns the first MQTTSN header and the `.payload` field returns the rest of the packet, 
which depends on the particular type of MQTTSN packet.
In this case, if you managed to sniff some MQTTSN packets, you might see something like:

```
<MQTTSN_CONNECT  dup=0 qos=0 retain=0 will=0 clean=1 topicIDtype=0 protocol=1 duration=30 client='client-temp' |>,
<MQTTSN_CONNACK  returnCode=0 |>,
 ```
 
These packets represent a connect request followed by an acknowledgment.

## 3. Sniffing in the background
Now let us write a script `sniff.py` to be running in the background. 
This script will collect 100 UDP packets silently, and dump the packet
trace into a log file `packet.log`.

In [3]:
num_packets = 100

with open('sniff.py', 'w') as file:
    file.write(
        'import logging\n')
    file.write(
        'logging.getLogger("scapy.runtime").setLevel(logging.ERROR)\n')
    file.write(
        'from scapy.all import *\n')
    file.write(
        'sniffer = sniff(iface="br0", filter=f"udp", \n')
    file.write(
        '                count={})\n'.format(num_packets))
    file.write(
        'print(sniffer.nsummary())\n')

os.system("chmod 777 sniff.py")
os.system("python3.6 sniff.py >packet.log&")

0

It may take some time for the background sniffing process to complete. 

Use the following command line to check whether the process 
has completed:
```
ps -aux | grep 'sniff.py'
```

After it is done, users can check the contents in `packet.log`.

Finally, remove all the generated files.

In [4]:
os.system("rm -rf sniff.py packet.log")

0